# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
# print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

The song_in_session table has artist, song, legnth, session_id, item_in_session columns.<br>
Partition key of this table is session_id, and clustering key is item_in_session.

In [8]:
table_name1 = "song_in_session"
drop_query = "DROP TABLE IF EXISTS %s" % table_name1

try:
    session.execute(drop_query)
except Exception as e:
    print(e)
create_query = """
CREATE TABLE IF NOT EXISTS %s 
(session_id int, 
item_in_session int, 
artist text, 
song text, 
length double,
PRIMARY KEY((session_id), item_in_session))""" % table_name1
try:
    session.execute(create_query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO %s (session_id, item_in_session, artist, song, length)" % table_name1
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[-3]), int(line[3]), line[0], line[-2], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "SELECT session_id, item_in_session, artist, song, length FROM %s WHERE session_id=338 AND item_in_session=4" % table_name1
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    
pd.DataFrame(rows)

,session_id,item_in_session,artist,song,length
0,338,4,Faithless,Music Matters (Mark Knight Dub),495.3073


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The song_in_user_and_session table has artist, song, first_name, last_name, user_id, session_id, item_in_session columns.<br>
Partition key of this table is user_id and session_id, and clustering key is item_in_session.

In [11]:
table_name2 = "song_in_user_and_session"
drop_query = "DROP TABLE IF EXISTS %s" % table_name2

try:
    session.execute(drop_query)
except Exception as e:
    print(e)
create_query = """
CREATE TABLE IF NOT EXISTS %s 
(user_id int,
session_id int, 
item_in_session int,
artist text, 
song text, 
first_name text,
last_name text,
PRIMARY KEY((user_id, session_id), item_in_session))""" % table_name2
try:
    session.execute(create_query)
except Exception as e:
    print(e)                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO %s (user_id, session_id, item_in_session, artist, song, first_name, last_name)" % table_name2
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[-1]), int(line[-3]), int(line[3]), line[0], line[-2], line[1], line[4]))

In [13]:
query = "SELECT user_id, session_id, item_in_session, artist, song, first_name, last_name FROM %s WHERE user_id=10 AND session_id=182" % table_name2
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
pd.DataFrame(rows)

,user_id,session_id,item_in_session,artist,song,first_name,last_name
0,10,182,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,10,182,1,Three Drives,Greece 2000,Sylvie,Cruz
2,10,182,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,10,182,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz



### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


The user_by_song table has song, user_id, first_name, last_name columns.<br>
Partition key of this table is song, and clustering key is user_id.

In [18]:
table_name3 = "user_by_song"
drop_query = "DROP TABLE IF EXISTS %s" % table_name3

try:
    session.execute(drop_query)
except Exception as e:
    print(e)
create_query = """
CREATE TABLE IF NOT EXISTS %s 
(song text, 
user_id int, 
first_name text,
last_name text,
PRIMARY KEY((song), user_id))""" % table_name3
try:
    session.execute(create_query)
except Exception as e:
    print(e)

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO %s (song, user_id, first_name, last_name)" % table_name3
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[-2], int(line[-1]), line[1], line[4]))

In [20]:
query = "SELECT song, first_name, last_name FROM %s WHERE song='All Hands Against His Own'" % table_name3
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
pd.DataFrame(rows)

,song,first_name,last_name
0,All Hands Against His Own,Jacqueline,Lynch
1,All Hands Against His Own,Tegan,Levine
2,All Hands Against His Own,Sara,Johnson


### Drop the tables before closing out the sessions

In [21]:
query = "drop table "
tables = [table_name1, table_name2, table_name3]
try:
    for table in tables:
        drop_query = query + table
        rows = session.execute(drop_query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()